# WEB SCRAPING-ASSIGNMENT-3

In [ ]:
Write a python program which searches all the product under a particular product from www.amazon.in. The
product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for
guitars.

In [5]:
import boto3
from botocore.config import Config

# Replace these with your own access and secret keys
ACCESS_KEY = 'your_access_key'
SECRET_KEY = 'your_secret_key'
ASSOCIATE_TAG = 'your_associate_tag'

def search_amazon_products(query):
    config = Config(
        region_name='us-east-1',
        signature_version='v4',
        retries={
            'max_attempts': 10,
            'mode': 'standard'
        }
    )
    client = boto3.client(
        'advertising',
        aws_access_key_id=ACCESS_KEY,
        aws_secret_access_key=SECRET_KEY,
        config=config
    )
    
    response = client.search_items(
        Keywords=query,
        ItemPage=1,
        Resources=[
            'ItemInfo.Title', 'Offers.Listings.Price'
        ],
        PartnerTag=ASSOCIATE_TAG,
        PartnerType='Associates',
        Marketplace='www.amazon.in'
    )
    
    items = response.get('Items', [])
    for item in items:
        title = item['ItemInfo']['Title']['DisplayValue']
        price = item['Offers']['Listings'][0]['Price']['Dis

ModuleNotFoundError: No module named 'boto3'

In [ ]:
2. In the above question, now scrape the following details of each product listed in first 3 pages of your search
results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then
scrape all the products available under that product name. Details to be scraped are: "Brand
Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and
“Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def get_product_details(soup):
    product_list = []
    products = soup.find_all('div', {'data-component-type': 's-search-result'})
    
    for product in products:
        brand_name = product.find('span', {'class': 'a-size-base-plus'}).text.strip() if product.find('span', {'class': 'a-size-base-plus'}) else '-'
        product_name = product.find('span', {'class': 'a-size-medium a-color-base a-text-normal'}).text.strip() if product.find('span', {'class': 'a-size-medium a-color-base a-text-normal'}) else '-'
        price = product.find('span', {'class': 'a-offscreen'}).text.strip() if product.find('span', {'class': 'a-offscreen'}) else '-'
        return_exchange = '-'
        expected_delivery = '-'
        availability = '-'
        product_url = 'https://www.amazon.in' + product.find('a', {'class': 'a-link-normal a-text-normal'})['href'] if product.find('a', {'class': 'a-link-normal a-text-normal'}) else '-'
        
        product_list.append({
            'Brand Name': brand_name,
            'Name of the Product': product_name,
            'Price': price,
            'Return/Exchange': return_exchange,
            'Expected Delivery': expected_delivery,
            'Availability': availability,
            'Product URL': product_url
        })
    
    return product_list

def scrape_amazon(query, pages=3):
    base_url = "https://www.amazon.in/s"
    all_products = []
    
    for page in range(1, pages + 1):
        params = {
            'k': query,
            'page': page
        }
        
        response = requests.get(base_url, headers=headers, params=params)
        if response.status_code != 200:
            print(f"Failed to retrieve page {page}")
            break
        
        soup = BeautifulSoup(response.content, 'html.parser')
        products = get_product_details(soup)
        all_products.extend(products)
        
        # Sleep to avoid being blocked by Amazon
        time.sleep(2)
    
    return all_products

def save_to_csv(data, filename='amazon_products.csv'):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)

if __name__ == "__main__":
    query = input("Enter the product to search for: ")
    products = scrape_amazon(query)
    save_to_csv(products)
    print(f"Saved {len(products)} products to amazon_products.csv")

Enter the product to search for: SOUP
Failed to retrieve page 1
Saved 0 products to amazon_products.csv


In [ ]:
. Write a python program to access the search bar and search button on images.google.com and scrape 10
images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’

In [9]:
import time
import requests
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
# Function to download images
def download_image(url, folder_path, image_name):
    try:
        img_data = requests.get(url).content
        with open(os.path.join(folder_path, image_name), 'wb') as handler:
            handler.write(img_data)
    except Exception as e:
        print(f"Could not download {url} - {e}")

# Function to perform the image scraping
def scrape_images(keyword, num_images=10):
    # Set up the browser
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # Run in headless mode
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    # Go to Google Images
    driver.get('https://images.google.com/')
    search_box = driver.find_element(By.NAME, 'q')
    search_box.send_keys(keyword)
    search_box.send_keys(Keys.RETURN)

    # Scroll and collect image URLs
    image_urls = set()
    while len(image_urls) < num_images:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Give time for images to load
        thumbnails = driver.find_elements(By.CSS_SELECTOR, 'img.rg_i')
        for img in thumbnails:
            try:
                img.click()
                time.sleep(2)
                images = driver.find_elements(By.CSS_SELECTOR, 'img.n3VNCb')
                for image in images:
                    src = image.get_attribute('src')
                    if src and 'http' in src:
                        image_urls.add(src)
                        if len(image_urls) >= num_images:
                            break
            except Exception as e:
                print(f"Error clicking image: {e}")
            if len(image_urls) >= num_images:
                break

    driver.quit()

    # Download the images
    folder_path = f"images/{keyword.replace(' ', '_')}"
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    for i, url in enumerate(image_urls):
        download_image(url, folder_path, f"{keyword.replace(' ', '_')}_{i + 1}.jpg")

    print(f"Downloaded {len(image_urls)} images for keyword: {keyword}")

if __name__ == "__main__":
    keywords = ['fruits', 'cars', 'Machine Learning', 'Guitar', 'Cakes']
    for keyword in keywords:
        scrape_images(keyword)

ModuleNotFoundError: No module named 'selenium'

In [ ]:
Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com
and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand
Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”,
“Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the
details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def get_smartphone_details(soup):
    smartphone_list = []
    products = soup.find_all('div', {'class': '_1AtVbE'})

    for product in products:
        try:
            brand_name = product.find('div', {'class': '_4rR01T'}).text.split(' ')[0] if product.find('div', {'class': '_4rR01T'}) else '-'
            smartphone_name = product.find('div', {'class': '_4rR01T'}).text if product.find('div', {'class': '_4rR01T'}) else '-'
            price = product.find('div', {'class': '_30jeq3 _1_WHN1'}).text if product.find('div', {'class': '_30jeq3 _1_WHN1'}) else '-'
            product_url = 'https://www.flipkart.com' + product.find('a', {'class': '_1fQZEK'})['href'] if product.find('a', {'class': '_1fQZEK'}) else '-'

            details = product.find('ul', {'class': '_1xgFaf'}).find_all('li') if product.find('ul', {'class': '_1xgFaf'}) else []
            color, ram, storage, primary_camera, secondary_camera, display_size, battery_capacity = '-', '-', '-', '-', '-', '-', '-'
            for detail in details:
                text = detail.text
                if 'Color' in text:
                    color = text.split(':')[1].strip()
                elif 'RAM' in text:
                    ram = text.split('|')[0].strip()
                    storage = text.split('|')[1].strip()
                elif 'Primary Camera' in text:
                    primary_camera = text.split('|')[0].strip()
                    secondary_camera = text.split('|')[1].strip()
                elif 'Display' in text:
                    display_size = text.split(':')[1].strip()
                elif 'Battery' in text:
                    battery_capacity = text.split(':')[1].strip()

            smartphone_list.append({
                'Brand Name': brand_name,
                'Smartphone Name': smartphone_name,
                'Colour': color,
                'RAM': ram,
                'Storage(ROM)': storage,
                'Primary Camera': primary_camera,
                'Secondary Camera': secondary_camera,
                'Display Size': display_size,
                'Battery Capacity': battery_capacity,
                'Price': price,
                'Product URL': product_url
            })
        except Exception as e:
            print(f"Error processing product: {e}")

    return smartphone_list

def scrape_flipkart_smartphones(query):
    base_url = "https://www.flipkart.com/search"
    params = {'q': query}
    response = requests.get(base_url, headers=headers, params=params)

    if response.status_code != 200:
        print(f"Failed to retrieve page: {response.status_code}")
        return []

    soup = BeautifulSoup(response.content, 'html.parser')
    smartphones = get_smartphone_details(soup)
    return smartphones

def save_to_csv(data, filename='flipkart_smartphones.csv'):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)

if __name__ == "__main__":
    query = input("Enter the smartphone to search for: ")
    smartphones = scrape_flipkart_smartphones(query)
    save_to_csv(smartphones)
    print(f"Saved {len(smartphones)} smartphones to flipkart_smartphones.csv")

Enter the smartphone to search for: BeautifulSoup
Failed to retrieve page: 500
Saved 0 smartphones to flipkart_smartphones.csv


In [ ]:
Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps. 

In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

def get_coordinates(city):
    # Set up the browser
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # Run in headless mode
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    # Go to Google Maps
    driver.get('https://maps.google.com')

    # Find the search box and enter the city name
    search_box = driver.find_element(By.ID, 'searchboxinput')
    search_box.send_keys(city)
    search_box.send_keys(Keys.RETURN)

    # Wait for the results to load
    time.sleep(5)  # Adjust the sleep time if needed

    # Extract the coordinates from the URL
    current_url = driver.current_url
    driver.quit()

    try:
        # Extract latitude and longitude from the URL
        if "/@-?[0-9]+" in current_url:
            coordinates = current_url.split('/@')[1].split(',')[0:2]
            latitude = coordinates[0]
            longitude = coordinates[1]
            return float(latitude), float(longitude)
        else:
            return None, None
    except Exception as e:
        print(f"Error extracting coordinates: {e}")
        return None, None

if __name__ == "__main__":
    city = input("Enter the city name to search for: ")
    latitude, longitude = get_coordinates(city)
    if latitude and longitude:
        print(f"Coordinates of {city}:")
        print(f"Latitude: {latitude}")
        print(f"Longitude: {longitude}")
    else:
        print(f"Could not find coordinates for {city}.")

ModuleNotFoundError: No module named 'selenium'

In [ ]:
Write a program to scrap all the available details of best gaming laptops from digit.in. 

In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_laptop_details(soup):
    laptop_list = []

    laptops = soup.find_all('div', class_='specs-box')

    for laptop in laptops:
        try:
            name = laptop.find('h3').text.strip()
            details = laptop.find_all('div', class_='value')
            specs = {
                'Laptop Name': name,
                'Processor': details[0].text.strip() if len(details) > 0 else '-',
                'Graphics': details[1].text.strip() if len(details) > 1 else '-',
                'RAM': details[2].text.strip() if len(details) > 2 else '-',
                'Storage': details[3].text.strip() if len(details) > 3 else '-',
                'Display': details[4].text.strip() if len(details) > 4 else '-',
                'Price': details[5].text.strip() if len(details) > 5 else '-'
            }
            laptop_list.append(specs)
        except Exception as e:
            print(f"Error processing laptop: {e}")

    return laptop_list

def scrape_digit_gaming_laptops():
    url = "https://www.digit.in/top-products/best-gaming-laptops-40.html"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print(f"Failed to retrieve page: {response.status_code}")
        return []

    soup = BeautifulSoup(response.content, 'html.parser')
    laptops = get_laptop_details(soup)
    return laptops

def save_to_csv(data, filename='digit_gaming_laptops.csv'):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)

if __name__ == "__main__":
    laptops = scrape_digit_gaming_laptops()
    save_to_csv(laptops)
    print(f"Saved {len(laptops)} laptops to digit_gaming_laptops.csv")


Saved 0 laptops to digit_gaming_laptops.csv


In [ ]:
Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped:
“Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”. 

In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_billionaire_details(soup):
    billionaire_list = []
    
    rows = soup.find_all('div', class_='personName')
    
    for row in rows:
        try:
            rank = row.find('div', class_='rank').text.strip() if row.find('div', class_='rank') else '-'
            name = row.find('div', class_='personName').text.strip() if row.find('div', class_='personName') else '-'
            net_worth = row.find('div', class_='netWorth').text.strip() if row.find('div', class_='netWorth') else '-'
            age = row.find('div', class_='age').text.strip() if row.find('div', class_='age') else '-'
            citizenship = row.find('div', class_='countryOfCitizenship').text.strip() if row.find('div', class_='countryOfCitizenship') else '-'
            source = row.find('div', class_='source').text.strip() if row.find('div', class_='source') else '-'
            industry = row.find('div', class_='category').text.strip() if row.find('div', class_='category') else '-'
            
            billionaire_list.append({
                'Rank': rank,
                'Name': name,
                'Net worth': net_worth,
                'Age': age,
                'Citizenship': citizenship,
                'Source': source,
                'Industry': industry
            })
        except Exception as e:
            print(f"Error processing row: {e}")

    return billionaire_list

def scrape_forbes_billionaires():
    url = "https://www.forbes.com/billionaires/"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print(f"Failed to retrieve page: {response.status_code}")
        return []

    soup = BeautifulSoup(response.content, 'html.parser')
    billionaires = get_billionaire_details(soup)
    return billionaires

def save_to_csv(data, filename='forbes_billionaires.csv'):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)

if __name__ == "__main__":
    billionaires = scrape_forbes_billionaires()
    save_to_csv(billionaires)
    print(f"Saved {len(billionaires)} billionaires to forbes_billionaires.csv")


Saved 0 billionaires to forbes_billionaires.csv


In [ ]:
Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted
from any YouTube Video.

In [14]:
from googleapiclient.discovery import build
import pandas as pd

# Define your API key
API_KEY = 'YOUR_API_KEY'

# YouTube video ID
VIDEO_ID = 'YOUR_VIDEO_ID'

# Create a build object
youtube = build('youtube', 'v3', developerKey=API_KEY)

def get_video_comments(video_id, max_results):
    comments = []
    page_token = None

    while len(comments) < max_results:
        # Get the video comments
        request = youtube.commentThreads().list(
            part='snippet',
            videoId=video_id,
            maxResults=100,
            pageToken=page_token,
            textFormat='plainText'
        )
        response = request.execute()

        # Parse the response
        for item in response['items']:
            comment = item['snippet']['topLevelComment']['snippet']
            comments.append({
                'Comment': comment['textDisplay'],
                'Upvotes': comment['likeCount'],
                'Time': comment['publishedAt']
            })

        # Check if there is a next page
        if 'nextPageToken' in response and len(comments) < max_results:
            page_token = response['nextPageToken']
        else:
            break

    return comments[:max_results]

def save_to_csv(data, filename='youtube_comments.csv'):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)

if __name__ == "__main__":
    max_results = 500
    comments = get_video_comments(VIDEO_ID, max_results)
    save_to_csv(comments)
    print(f"Saved {len(comments)} comments to youtube_comments.csv")

ModuleNotFoundError: No module named 'googleapiclient'

In [ ]:
Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in
“London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall
reviews, privates from price, dorms from price, facilities and property description. 

In [16]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL for hostels in London
url = "https://www.hostelworld.com/hostels/London"

# Request the webpage
response = requests.get(url)
response.raise_for_status()

# Parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')

# Function to extract hostel details
def get_hostel_details(hostel):
    name = hostel.find('h2', class_='title').text.strip() if hostel.find('h2', class_='title') else '-'
    distance = hostel.find('span', class_='distance-description').text.strip() if hostel.find('span', class_='distance-description') else '-'
    rating = hostel.find('div', class_='score orange big').text.strip() if hostel.find('div', class_='score orange big') else '-'
    total_reviews = hostel.find('div', class_='reviews').text.strip() if hostel.find('div', class_='reviews') else '-'
    overall_reviews = hostel.find('div', class_='keyword').text.strip() if hostel.find('div', class_='keyword') else '-'
    privates_from_price = hostel.find('div', class_='price privates').text.strip() if hostel.find('div', class_='price privates') else '-'
    dorms_from_price = hostel.find('div', class_='price dorms').text.strip() if hostel.find('div', class_='price dorms') else '-'
    facilities = ', '.join([fac.text.strip() for fac in hostel.find_all('div', class_='facilities-label facility')])
    description = hostel.find('div', class_='rating-fab-text').text.strip() if hostel.find('div', class_='rating-fab-text') else '-'
    
    return {
        'Hostel Name': name,
        'Distance from City Centre': distance,
        'Ratings': rating,
        'Total Reviews': total_reviews,
        'Overall Reviews': overall_reviews,
        'Privates From Price': privates_from_price,
        'Dorms From Price': dorms_from_price,
        'Facilities': facilities,
        'Property Description': description
    }

# Extract hostel information
hostel_list = []
hostels = soup.find_all('div', class_='fabresult')
for hostel in hostels:
    details = get_hostel_details(hostel)
    hostel_list.append(details)

# Save the data to a CSV file
def save_to_csv(data, filename='hostelworld_london_hostels.csv'):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)

if __name__ == "__main__":
    save_to_csv(hostel_list)
    print(f"Saved {len(hostel_list)} hostels to hostelworld_london_hostels.csv")


HTTPError: 404 Client Error: Not Found for url: https://www.hostelworld.com/hostels/London